## Capstone Project - The Battle of Neighbourhoods

### Introduction/Business Problem

Ollie's Ice Cream + Stuff is a dog-friendly ice cream parlour based in Bushwick, New York City. They are the first-ever ice cream parlour in NYC making flavours dogs can actually eat.

With business booming, the owners of Ollie's Ice Cream + Stuff are wondering if they should open a franchise outlet in Toronto.

This report aims to find out: 
Should Ollie's Ice Cream + Stuff open a franchise outlet in Toronto? Which neighbourhood in Toronto is best suited for their store?

### Data

##### First, we need to find out if Ollie's Ice Cream + Stuff is likely to be as successful in Toronto as it is in New York City. 

We can do this by answering the following questions:

1. Does Toronto have an ice cream culture? 
2. Are there as many dog owners in Toronto as there are in New York City?

Using Foursquare location data, we can determine and compare the number of ice cream parlours in each city. 

Using the NYC Dog Licensing Dataset and the Toronto Dog Licence Database, we can determine and compare the number of dog owners in each city. 

##### Next, we need to find out which neighbourhood in Toronto is most similar to Bushwick.

We can do this by answering the following questions:

1. What amenities are available in Bushwick?
2. Which neighbourhood in Toronto has similar amenities?

Using Foursquare location data, we can determine and compare the amenities available in Bushwick and the different neighbourhoods in Toronto.